<a href="https://colab.research.google.com/github/alantang1369/Headline_Sarcasm/blob/main/Sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import tensorflow as tf
import numpy as np
import urllib

In [ ]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
urllib.request.urlretrieve(url, 'sarcasm.json')



('sarcasm.json', <http.client.HTTPMessage at 0x7cca4011cfa0>)

In [9]:
import pandas as pd
data = pd.read_json('sarcasm.json')

In [10]:
data.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [11]:
sentences = data['headline'].to_numpy()
labels = data['is_sarcastic'].to_numpy()

In [12]:
sentences[:5]

array(["former versace store clerk sues over secret 'black code' for minority shoppers",
       "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
       "mom starting to fear son's web series closest thing she will have to grandchild",
       'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
       'j.k. rowling wishes snape happy birthday in the most magical way'],
      dtype=object)

In [18]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000


In [13]:
from sklearn.model_selection import  train_test_split
train_sentences , val_sentences , train_labels , val_labels = train_test_split(sentences,
                                                                               labels,
                                                                               test_size = 0.1 ,
                                                                               random_state = 42)

In [15]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(24038, 24038, 2671, 2671)

In [26]:
max_length = round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))
max_length

10

In [28]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
text_vectorizer = TextVectorization(max_tokens = vocab_size,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)
text_vectorizer.adapt(train_sentences)

In [29]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=vocab_size, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1")

In [30]:
model = tf.keras.Sequential([

    layers.Input(shape=(10,), dtype='int64'),
    embedding,
    layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

history = model.fit(text_vectorizer(train_sentences),
                    train_labels,
                    epochs=10,
                    validation_data=(text_vectorizer(val_sentences), val_labels),
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])

Epoch 1/10
752/752 [==============================] - 7s 9ms/step - loss: 0.4912 - accuracy: 0.7702 - val_loss: 0.4183 - val_accuracy: 0.8124
Epoch 2/10
752/752 [==============================] - 3s 4ms/step - loss: 0.3889 - accuracy: 0.8237 - val_loss: 0.4118 - val_accuracy: 0.8165
Epoch 3/10
752/752 [==============================] - 2s 3ms/step - loss: 0.3786 - accuracy: 0.8274 - val_loss: 0.4111 - val_accuracy: 0.8124
Epoch 4/10
752/752 [==============================] - 3s 3ms/step - loss: 0.3756 - accuracy: 0.8296 - val_loss: 0.4127 - val_accuracy: 0.8102
Epoch 5/10
752/752 [==============================] - 2s 3ms/step - loss: 0.3739 - accuracy: 0.8309 - val_loss: 0.4121 - val_accuracy: 0.8121
Epoch 6/10
752/752 [==============================] - 2s 3ms/step - loss: 0.3735 - accuracy: 0.8307 - val_loss: 0.4120 - val_accuracy: 0.8143
Epoch 7/10
752/752 [==============================] - 2s 3ms/step - loss: 0.3729 - accuracy: 0.8310 - val_loss: 0.4115 - val_accuracy: 0.8132
Epoch 

In [32]:
embedding_2 = layers.Embedding(input_dim=vocab_size, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_2")

In [33]:
model_2a = tf.keras.Sequential([
    layers.Input(shape=(1,), dtype='string'),
    text_vectorizer,
    embedding_2,
    # x = layers.GRU(64, return_sequences=True)
    layers.Bidirectional(layers.LSTM(64, return_sequences=True)), # stacking RNN layers requires return_sequences=True
    layers.Bidirectional(layers.LSTM(64)),
    #layers.LSTM(64),
    layers.Dense(1, activation='sigmoid')
])

model_2a.compile(loss='binary_crossentropy',
                 optimizer=tf.keras.optimizers.Adam(),
                 metrics=['accuracy'])

model_2a.fit(train_sentences,
             train_labels,
             epochs=10,
             validation_data=(val_sentences, val_labels))

Epoch 1/10
752/752 [==============================] - 18s 16ms/step - loss: 0.4229 - accuracy: 0.7969 - val_loss: 0.3900 - val_accuracy: 0.8203
Epoch 2/10
752/752 [==============================] - 6s 9ms/step - loss: 0.3570 - accuracy: 0.8397 - val_loss: 0.3767 - val_accuracy: 0.8270
Epoch 3/10
752/752 [==============================] - 6s 8ms/step - loss: 0.3389 - accuracy: 0.8496 - val_loss: 0.3710 - val_accuracy: 0.8319
Epoch 4/10
752/752 [==============================] - 7s 9ms/step - loss: 0.3208 - accuracy: 0.8573 - val_loss: 0.3696 - val_accuracy: 0.8315
Epoch 5/10
752/752 [==============================] - 6s 8ms/step - loss: 0.3002 - accuracy: 0.8647 - val_loss: 0.3749 - val_accuracy: 0.8349
Epoch 6/10
752/752 [==============================] - 6s 8ms/step - loss: 0.2822 - accuracy: 0.8735 - val_loss: 0.3939 - val_accuracy: 0.8326
Epoch 7/10
752/752 [==============================] - 6s 8ms/step - loss: 0.2647 - accuracy: 0.8796 - val_loss: 0.4155 - val_accuracy: 0.8315
Epoc